In [1]:
import torch
import string
import re
import numpy as np

In [2]:
scheme = torch.load('contraction/640G_reproduce_scheme_n53_m20_ABCDCDAB_3000000_einsum_10_open_3.pt')

In [6]:
nsch = []
flat = [1] * 455
laten = -1
lasplit = -1
split = 512
for num,step in enumerate(scheme[1]):
    nstep = {}
    
    i, j = step[0]
    restletters = "".join(list((set(string.ascii_letters) - set(step[1]))))
    s1 = restletters[:flat[i]]
    s2 = restletters[flat[i]:flat[i] + flat[j]]
    ein_old = re.split('->|,', step[1])
    nstep1 = step[1]
    nstep1 = nstep1.replace(ein_old[0][0], s1)
    nstep1 = nstep1.replace(ein_old[1][0], s2)
    ein_new = re.split('->|,', nstep1)
    
    batch_i, batch_j = step[2]
    
    nstep['index'] = step[0]
    nstep['ein'] = nstep1
    nstep['split'] = 1
    
    if len(batch_i) > 1 or (len(step) > 3 and len(batch_i) == len(batch_j) == 1):
        bi = torch.cat(batch_i)
        bj = torch.cat(batch_j)
        blen = len(bi)
        if i == laten:
            laten = -1
            print('1 st')
            # for index in range(blen):
            #     bi[index] = labatch[bi[index]]
            bi = labatch[bi]
        
        nstep['type'] = 1
        nstep['flat'] = (flat[i], flat[j])
        print(num,' ',i,' ',j,' ',nstep['flat'])
        nstep['ein'] = step[1]
        if lasplit == -1:
            lasplit = i
            chubi_begin = list(range(0, bi[-1],  int(np.ceil(bi[-1]/split))))
            assert len(chubi_begin) == split, len(chubi_begin)
        assert lasplit == i
        print(chubi_begin)
        chubi = []
        chubj = []
        begin = 0
        chubi_begin_new = []
        for index in range(split - 1):
            chubi_begin_new.append(begin)
            valuemin = chubi_begin[index]
            valuemax = chubi_begin[index+1]
            end = torch.searchsorted(bi, valuemax)
            print(begin,end, begin-end,bi[begin:end]-valuemin)
            chubi.append(bi[begin:end]-valuemin)
            chubj.append(bj[begin:end])
            begin = end
        chubi_begin_new.append(begin)
        chubi_begin = chubi_begin_new
        chubi.append(bi[begin:] - valuemax)
        chubj.append(bj[begin:])
        nstep['chunk_batch'] = (chubi, chubj)
        flat[i] = 1
    elif len(step)>3:
        nstep['flat'] = (flat[i], flat[j])
        print(num,' ',i,' ',j,' ',nstep['flat'])
        flat[i] = flat[i] + flat[j]
        nstep['type'] = 2
        if len(batch_i) == 1:
            laten = i
            labatch = batch_i[0]
    else:
        nstep['flat'] = (flat[i], flat[j])
        print(num,' ',i,' ',j,' ',nstep['flat'])
        assert flat[i] == 1 or flat[j] == 1
        if flat[i] > 1:
            assert ein_old[2][0] == ein_old[0][0]
            flat[i] = flat[i]
        elif flat[j] > 1:
            assert ein_old[2][0] == ein_old[1][0]
            flat[i] = flat[j]
        nstep['type'] = 3
    nsch.append(nstep)

0   27   9   (1, 1)
1   27   5   (1, 1)
2   47   70   (1, 1)
3   27   47   (1, 1)
4   71   90   (1, 1)
5   27   71   (1, 1)
6   35   55   (1, 1)
7   35   78   (1, 1)
8   79   98   (1, 1)
9   35   79   (1, 1)
10   35   13   (1, 1)
11   27   35   (1, 1)
12   51   31   (1, 1)
13   51   74   (1, 1)
14   51   75   (1, 1)
15   94   117   (1, 1)
16   51   94   (1, 1)
17   32   52   (1, 1)
18   51   32   (1, 1)
19   136   160   (1, 1)
20   136   179   (1, 1)
21   201   221   (1, 1)
22   201   202   (1, 1)
23   136   201   (1, 1)
24   51   136   (1, 1)
25   113   140   (1, 1)
26   121   164   (1, 1)
27   113   121   (1, 1)
28   175   197   (1, 1)
29   132   175   (1, 1)
30   132   156   (1, 1)
31   132   183   (1, 1)
32   113   132   (1, 1)
33   51   113   (1, 1)
34   27   51   (1, 1)
35   137   152   (1, 1)
36   137   110   (1, 1)
37   161   180   (1, 1)
38   137   161   (1, 1)
39   118   143   (1, 1)
40   137   118   (1, 1)
41   76   95   (1, 1)
42   137   76   (1, 1)
43   27   137   (1, 1)
4

In [7]:
stemindex = 27
permute = {stemindex:np.arange(len(nsch[-1]['ein'].split('->')[1])).tolist()}
for nstep in range(len(nsch)-1,-1,-1):
    i, j = nsch[nstep]['index']
    print('i=',i)
    ein = re.split('->|,', nsch[nstep]['ein'])
    print(nstep, ein,permute[i])
    ein[2] = "".join([ein[2][t] for t in permute[i]])
    permute[i] = list(range(len(ein[0])))
    permute[j] = list(range(len(ein[1])))
    if len(ein[0])>=5:
        permute[i] = np.argsort([ein[2].find(x) if x in ein[2] else 100+ein[0].find(x) for x in ein[0]])
        
    if len(ein[1])>=5:
        permute[j] = np.argsort([ein[2].find(x) if x in ein[2] else 100+ein[0].find(x) for x in ein[1]])
        
    ein[0] = "".join([ein[0][t] for t in permute[i]])
    ein[1] = "".join([ein[1][t] for t in permute[j]])
    
    flati, flatj = nsch[nstep]['flat']
    if nsch[nstep]['type'] == 1:
        if flati > 1:
            assert permute[i][0] == 0
            permute[i] = list(range(flati)) + [x + flati - 1 for x in permute[i][1:]]
        if flatj > 1:
            assert permute[j][0] == 0
            permute[j] = list(range(flatj)) + [x + flatj - 1 for x in permute[j][1:]]
    
    nsch[nstep]['ein_2'] = f'{ein[0]},{ein[1]}->{ein[2]}'
    print(nsch[nstep]['ein_2'])

i= 27
443 ['MFEDBCAGHKIJ', 'MEFL', 'MDBCAGHKIJL'] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
MDBCAGHKIJFE,MEFL->MDBCAGHKIJL
i= 385
442 ['rBCD', 'aD', 'arBC'] [0, 1, 2, 3]
rBCD,aD->arBC
i= 27
441 ['QKCGDFJHIBEA', 'QFGLMPDCHNO', 'QKJIBEALMPNO'] [ 0  3  4  5  6  7  8  9 10 11  1  2]
QIBEAKJCGDFH,QLMPNOCGDFH->QIBEALMPNOKJ
i= 397
440 ['rCDGHKFBA', 'aFIJ', 'rCDGHKBAaIJ'] [ 0  3  4  5  9 10  7  2  6  1  8]
rGHKADBCF,aFIJ->rGHKIJADBCa
i= 397
439 ['rCDGHFI', 'aFBA', 'rCDGHIaBA'] [0 3 4 5 8 2 7 1 6]
rGHIDCF,aFBA->rGHIADBCa
i= 397
438 ['rBDE', 'aCF', 'arBDECF'] [0 3 4 6 2 1 5]
rBDE,aCF->aDEFBrC
i= 400
437 ['rAD', 'aD', 'arA'] [0, 1, 2]
rAD,aD->arA
i= 27
436 ['TAPGLRSNDOMKB', 'TAPCMGKQBEFSIJLH', 'TRNDOCQEFIJH'] [ 0  8  9 10 11  1  6  2  3  4  5  7]
TRNDOAPGLSMKB,TFIJHQCEAPGLSMKB->TFIJHRQNDOCE
i= 330
435 ['raYXiceAJPTCNFOKQRMB', 'nWDJOERUHIMLTG', 'raYXicenWAPCNFKQBDEUHILG'] [0, 1, 2, 3, 4, 5, 6, 7, 8, 18, 20, 21, 23, 15, 11, 17, 9, 10, 13, 22, 19, 12, 14, 16]
raYXiceQCAPFNKBJTORM,nWEHIGDLUJTORM->raYXicenWE

In [8]:
logmodelife = {}
prestep = {}
lastmodelife = []
tmp = None
for i in range(54,357,1):
    if stemindex not in nsch[i]['index']:
        continue
    ein = re.split('->|,', nsch[i]['ein_2'])
    print(i, ein)
    if lastmodelife == []:
        lastmodelife = [0] * len(ein[2])
    modelife = []
    for c in ein[2]:
        if c in ein[0]:
            pos = ein[0].find(c)
            modelife.append(lastmodelife[pos] + 1)
        else:
            modelife.append(0)
    lastmodelife = modelife
    logmodelife[i] = modelife
    prestep[i] = tmp
    tmp = i
    print(np.sort(modelife)[-3:], modelife)

54 ['YCdMOXPbQANRFcDarVfiIS', 'ETBeJgULHWrVfiIS', 'ETBYCeJdMOgXPUbQANRFLcDaHW']
[1 1 1] [0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0]
60 ['FVBbCiKgrQjaSWdTAOUHLfDcZX', 'GYEJePRhNfDcZX', 'FVBGbCiKgrQjYaESWJdTAeOUHLPRhN']
[2 2 2] [1, 1, 1, 0, 2, 2, 1, 1, 2, 2, 2, 1, 0, 2, 0, 2, 1, 0, 2, 2, 2, 0, 2, 2, 2, 1, 0, 0, 0, 0]
80 ['EbBFfCnOirUodeDXcMgYAhSaGPTWjR', 'ZlkKLIJNVHSaGPTWjR', 'EbBFfCnOirUodeDZlXkKLIJcMNgYAhVH']
[3 3 3] [2, 2, 2, 1, 3, 3, 2, 2, 3, 3, 3, 2, 1, 3, 1, 0, 0, 3, 0, 0, 0, 0, 0, 2, 1, 0, 3, 3, 3, 1, 0, 0]
84 ['FXCGcDlQgrTmabEjiVhMNKLYOPeWAdUJ', 'HSIZBkeWAdUJ', 'FXCGcDHlSQgIrTmaZBbEjiVhMNKLkYOP']
[4 4 4] [3, 3, 3, 2, 4, 4, 0, 3, 0, 3, 4, 0, 4, 4, 3, 2, 0, 0, 4, 2, 1, 1, 4, 1, 1, 1, 1, 1, 0, 3, 2, 1]
86 ['FVBGbCHiSQdIrTjYcAaDgfUeMNKLhWZP', 'EJXWZP', 'FVBGEJbCHiSXQdIrTjYcAaDgfUeMNKLh']
[5 5 5] [4, 4, 4, 3, 0, 0, 5, 5, 1, 4, 1, 0, 4, 5, 1, 5, 5, 4, 3, 1, 1, 5, 3, 2, 2, 5, 2, 2, 2, 2, 2, 1]
101 ['GYBHFKeDInSaQgJrTobfAcEkjZiOPMNl', 'UhCXmLdWpZiOPMNl'

In [9]:
mgmodes = 3
out_selmode = None
lastorder = None
for nstep in range(356,53,-1):
    if nstep not in logmodelife.keys():
        continue
    ein_old = re.split('->|,', nsch[nstep]['ein_2'])
    ein_new = re.split('->|,', nsch[nstep]['ein_2'])
    stepmodelife = torch.tensor(logmodelife[nstep])
    if prestep[nstep]!=None:
        prestepmodelife = torch.tensor(logmodelife[prestep[nstep]])  
    if out_selmode == None:      # 起始
        outmgchar = [ein_old[2][i] for i in range(3)]
        in_selmode = [ein_old[0].find(c) for c in outmgchar]
        print(in_selmode)
        in_selmode_out = sorted(in_selmode)
        
        new_order = in_selmode + [i for i in range(len(ein_old[0])) if i not in in_selmode]
        ein_new[0] = np.array(list(ein_new[0]))[new_order]
        ein_new[0] = "".join(ein_new[0].tolist())
    elif prestep[nstep]==None: # 结束
        in_selmode = None
        
        ein_new[2] = np.array(list(ein_new[2]))[lastorder]        
        ein_new[2] = "".join(ein_new[2].tolist())
        
    elif stepmodelife[out_selmode].min() > 1: #不需要重排
        mgchar = [ein_old[2][i] for i in out_selmode]
        in_selmode = [ein_old[0].find(c) for c in mgchar]

        ein_new[2] = np.array(list(ein_new[2]))[lastorder]        
        ein_new[2] = "".join(ein_new[2].tolist())
        
        new_order = in_selmode + [i for i in range(len(prestepmodelife)) if i not in in_selmode]
        ein_new[0] = np.array(list(ein_new[0]))[new_order]     
        ein_new[0] = "".join(ein_new[0].tolist())
        
    else: #重排
        outmgchar = [ein_old[2][i] for i in out_selmode]
        in_selmode_out = [ein_old[0].find(c) for c in outmgchar]
        for x in in_selmode_out:
            prestepmodelife[x] = 0 
        in_selmode = prestepmodelife.argsort(descending=True,stable=True)[:mgmodes]
        in_selmode = in_selmode.sort().values.tolist()
        print((in_selmode_out), (in_selmode))
        assert len(set(in_selmode_out) &set(in_selmode)) == 0
        
        ein_new[2] = np.array(list(ein_new[2]))[lastorder]        
        ein_new[2] = "".join(ein_new[2].tolist())
        
        new_order = in_selmode + in_selmode_out + \
                    [i for i in range(len(prestepmodelife)) if i not in (in_selmode + in_selmode_out)]
        ein_new[0] = np.array(list(ein_new[0]))[new_order]     
        ein_new[0] = "".join(ein_new[0].tolist())
        
        
    print(nstep,stepmodelife[out_selmode], nsch[nstep]['flat'], nsch[nstep]['type'])
    print(f"{ein_new[0]},{ein_new[1]}->{ein_new[2]}")
    print(f"{ein_old[0]},{ein_old[1]}->{ein_old[2]}")
    nsch[nstep]['reorder_ein'] = f"{ein_new[0]},{ein_new[1]}->{ein_new[2]}"
    out_selmode = in_selmode
    lastorder = new_order
        
        

[0, 1, 2]
356 tensor([[ 6,  4,  4,  3,  3,  2,  2,  2,  2,  1,  1,  1,  1,  1,  1,  0,  0,  0,
          0,  3, 10,  5,  5,  4,  7,  4,  0, 10,  7,  5,  3,  3,  2,  2,  0]]) (15, 4) 2
raYicenjyltupwvUBTGODIRLPAHXFMKJEQN,fzZgCSMKJEQN->raYicenjyltupwvfzZgUBTGODICRLPAHXFS
raYicenjyltupwvUBTGODIRLPAHXFMKJEQN,fzZgCSMKJEQN->raYicenjyltupwvfzZgUBTGODICRLPAHXFS
348 tensor([5, 3, 3]) (9, 6) 2
rienjyltuaBYITCLWOUAJDHPNMEVKGXSRFd,pwvfzZQKGXSRFd->rienjyltupwvfzZaBYITCLWOUAJDHPNMEVQ
rienjyltuaBYITCLWOUAJDHPNMEVKGXSRFd,pwvfzZQKGXSRFd->rienjyltupwvfzZaBYITCLWOUAJDHPNMEVQ
337 tensor([4, 2, 2]) (5, 4) 2
rZltugBfIXCMcQYAJRONELGdUSFKPaeiVTW,xsnqDHbkKPaeiVTW->rZltuxsnqgBfIXCMcQYAJDHRONEbLGdUSFk
rZltugBfIXCMcQYAJRONELGdUSFKPaeiVTW,xsnqDHbkKPaeiVTW->rZltuxsnqgBfIXCMcQYAJDHRONEbLGdUSFk
[0, 1, 2] [3, 16, 28]
328 tensor([3, 1, 1]) (3, 2) 2
BLKrZtiGaCMeQbRONDFfVTJPchmWUdSgHYk,uxjAIEXKdSgHYk->rZtuxjBiGaCMeQbAIRONDLFfVTEJPchmWUX
rZtBiGaCMeQbRONDLFfVTJPchmWUKdSgHYk,uxjAIEXKdSgHYk->rZtuxjBiGaCMeQbAIRONDLFfVTEJPchmW

In [10]:
import torch
torch.save(nsch,f'nsch640_split{split}_mg3_open10.pt')